In [29]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('basics').getOrCreate()

# 02 - Data Understanding

In [30]:
# Load datasets
df_1 = spark.read.csv('Sleep_health_and_lifestyle_dataset.csv', header=True)
df_2 = spark.read.csv('Sleep_health_and_lifestyle_dataset_2.csv', header=True)

# Explore data
# Initial inspection of data
df_1.show()
df_2.show()


+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
|Person ID|Gender|Age|          Occupation|Sleep Duration|Quality of Sleep|Physical Activity Level|Stress Level| BMI Category|Blood Pressure|Heart Rate|Daily Steps|Sleep Disorder|
+---------+------+---+--------------------+--------------+----------------+-----------------------+------------+-------------+--------------+----------+-----------+--------------+
|        1|  Male| 27|   Software Engineer|           6.1|             3.5|                     42|           6|   Overweight|        126/83|        77|       5200|          None|
|        2|  Male| 28|              Doctor|           6.2|               4|                     82|           8|       Normal|        125/80|        75|       8000|          None|
|        3|  Male| 28|              Doctor|           6.2|               5|                     67| 

In [31]:
# Getting number of rows and columns for datasets
print((df_1.count(), len(df_1.columns)))
print((df_2.count(), len(df_2.columns)))

(374, 13)
(374, 5)


In [32]:
# Columns labels
print(df_1.columns)
print(df_2.columns)

['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Stress Level', 'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps', 'Sleep Disorder']
['Person ID', 'Bedtime', 'PhoneUse', 'WearsGlasses', 'Religious']


In [33]:
# Description of data
df_1.describe().show(truncate=False, vertical=True)
df_2.describe().show()

-RECORD 0-------------------------------------
 summary                 | count              
 Person ID               | 374                
 Gender                  | 374                
 Age                     | 374                
 Occupation              | 374                
 Sleep Duration          | 374                
 Quality of Sleep        | 374                
 Physical Activity Level | 374                
 Stress Level            | 374                
 BMI Category            | 374                
 Blood Pressure          | 374                
 Heart Rate              | 374                
 Daily Steps             | 374                
 Sleep Disorder          | 374                
-RECORD 1-------------------------------------
 summary                 | mean               
 Person ID               | 187.5              
 Gender                  | null               
 Age                     | 42.18449197860963  
 Occupation              | null               
 Sleep Durati

In [38]:
# Let's import in the relevant types.

from pyspark.sql.types import (StructField,StringType,IntegerType,FloatType,StructType)

# Correct structure for now
data_schema = [StructField('PersonID',FloatType(),True),
              StructField('Gender',StringType(),True),
              StructField('Age',FloatType(),True),
              StructField('Occupation',StringType(),True),
              StructField('Sleep Duration',FloatType(),True),
              StructField('Quality of Sleep',FloatType(),True),
              StructField('Physical Activity Level',FloatType(),True),
              StructField('Stress Level',FloatType(),True),
              StructField('BMI Category',StringType(),True),
              StructField('Blood Pressure',StringType(),True),
              StructField('Heart Rate',FloatType(),True),
              StructField('Daily Steps',FloatType(),True),
              StructField('Sleep Disorder',StringType(),True),
              ]

final_struct = StructType(fields=data_schema)
# Apply schema to df_1 with new variable types
df_1 = spark.read.csv('Sleep_health_and_lifestyle_dataset.csv', schema=final_struct)

df_1.printSchema()


root
 |-- PersonID: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Sleep Duration: float (nullable = true)
 |-- Quality of Sleep: float (nullable = true)
 |-- Physical Activity Level: float (nullable = true)
 |-- Stress Level: float (nullable = true)
 |-- BMI Category: string (nullable = true)
 |-- Blood Pressure: string (nullable = true)
 |-- Heart Rate: float (nullable = true)
 |-- Daily Steps: float (nullable = true)
 |-- Sleep Disorder: string (nullable = true)



In [41]:
# Only looking at age.show()
df_1.select('Age').describe().show()


+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|              374|
|   mean|42.18449197860963|
| stddev|8.673133465547242|
|    min|             27.0|
|    max|             59.0|
+-------+-----------------+



In [47]:
# Looking at summary of Physical Activity Level and Daily Steps
df_1.select('Physical Activity Level').describe().show()
df_1.dropna(subset=['Daily Steps']).select('Daily Steps').describe().show()

+-------+-----------------------+
|summary|Physical Activity Level|
+-------+-----------------------+
|  count|                    374|
|   mean|     47.094652406512736|
| stddev|     25.761316745976746|
|    min|                    0.3|
|    max|                  210.0|
+-------+-----------------------+

+-------+------------------+
|summary|       Daily Steps|
+-------+------------------+
|  count|               368|
|   mean| 5839.942934782609|
| stddev|1769.0999572833714|
|    min|            2000.0|
|    max|           14000.0|
+-------+------------------+



In [44]:
# Number of unique values
print (df_1.select('Physical Activity Level').distinct().count())
print (df_1.select('Daily Steps').distinct().count())

68
63


In [51]:
# Count of values for each occupation
df_1.groupBy('Occupation').count().show()

+--------------------+-----+
|          Occupation|count|
+--------------------+-----+
|           Scientist|    4|
|          Occupation|    1|
|               Nurse|   73|
|         Salesperson|   32|
|              Lawyer|   47|
|             Teacher|   40|
|Sales Representative|    2|
|              Doctor|   71|
|            Engineer|   63|
|          Accountant|   37|
|             Manager|    1|
|   Software Engineer|    4|
+--------------------+-----+



In [58]:
# Distinct values for Bedtime
print (df_2.select('Bedtime').distinct().count())


42


In [52]:
# Get values with decimal points (not .0) in Quality of Sleep
df_1.filter(col('Quality of Sleep') % 1 != 0).show()

+--------+------+----+-----------------+--------------+----------------+-----------------------+------------+------------+--------------+----------+-----------+--------------+
|PersonID|Gender| Age|       Occupation|Sleep Duration|Quality of Sleep|Physical Activity Level|Stress Level|BMI Category|Blood Pressure|Heart Rate|Daily Steps|Sleep Disorder|
+--------+------+----+-----------------+--------------+----------------+-----------------------+------------+------------+--------------+----------+-----------+--------------+
|     1.0|  Male|27.0|Software Engineer|           6.1|             3.5|                   42.0|         6.0|  Overweight|        126/83|      77.0|     5200.0|          None|
|    91.0|  Male|35.0|         Engineer|           7.3|             7.5|                   60.0|         4.0|      Normal|        125/80|      65.0|     5000.0|          None|
|   134.0|Female|38.0|       Accountant|           7.1|             7.5|                   60.0|         4.0|      Norma

In [56]:
df_1.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_1.columns]).show(truncate=False, vertical=True)


-RECORD 0----------------------
 PersonID                | 1   
 Gender                  | 0   
 Age                     | 1   
 Occupation              | 0   
 Sleep Duration          | 1   
 Quality of Sleep        | 1   
 Physical Activity Level | 1   
 Stress Level            | 1   
 BMI Category            | 0   
 Blood Pressure          | 0   
 Heart Rate              | 1   
 Daily Steps             | 1   
 Sleep Disorder          | 0   

